## New-York Taxi Оформление проекта

Рассмотрим временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

In [15]:
from ipywidgets import interact
import pandas as pd

from matplotlib import pyplot as plt

import folium

In [2]:
full_data =pd.read_csv("full_groupped_small.csv", sep=';') 
regions =pd.read_csv("regions.csv", sep=';') 
full_data['datetime'] = pd.to_datetime(full_data['datetime'])

In [4]:
full_data = full_data[ (full_data['datetime'] >= '2016-06-01 00:00:00') & (full_data['datetime'] <= '2016-06-30 17:00:00')]


In [5]:
days_to_choose = full_data.datetime.dt.strftime('%Y-%m-%d').unique()
time_to_choose = full_data.datetime.dt.strftime('%H:00:00').unique()

In [72]:
str(region) + '_' + dt2+'_'+ str(hour) +'_1'

'1075_2016-06-01_0_1'

In [73]:
# get List of prediction for 6 hours from file u need
def getListOfPredictions(region, date, hour ):
    
    dt = pd.to_datetime(date).strftime('%Y_%m_%d')
    dt2 = pd.to_datetime(date).strftime('%Y-%m-%d')
    l = [
        str(region) + '_' +dt2+'_'+ str(hour) +'_1',
        str(region) + '_' +dt2+'_'+ str(hour)+'_2',
        str(region) + '_' +dt2+'_'+ str(hour)+'_3',
        str(region) + '_' +dt2+'_'+ str(hour)+'_4',
        str(region) + '_' +dt2+'_'+ str(hour)+'_5',
        str(region) + '_' +dt2+'_'+ str(hour)+'_6',
    ]
    
    if (hour < 10) :
        hour =  '0' + str(hour)
    hour = str(hour)
    
    filename = dt + '_' + hour + '.csv'
    predict =pd.read_csv('predict/' + filename, header=None) 
    result = predict[predict[0].isin(l)]
    
    return result
    


In [88]:
def f(region_id, date, time):
    region_row =  regions[regions.region==region_id]
    choose_hour = pd.to_datetime(date + ' ' + time)
    end_hour = choose_hour + pd.DateOffset(hours=6)
    real_data = full_data[(full_data.region==region_id) & (full_data.datetime==choose_hour)]
    real_data_from_day = full_data[
        (full_data.region==region_id) & 
        (full_data.datetime>=choose_hour) &
        (full_data.datetime<end_hour)
    ]
    predictions = getListOfPredictions( region_id, date, choose_hour.hour ) 
    
    print(u"Реальный спрос на 2016-05-10 20:00:00: ", real_data.counts.values[0])
    print(u"Прогнозируемый спрос на 2016-05-10 20:00:00: ", predictions[1].values[0])
    west = region_row.west.values[0] 
    south = region_row.south.values[0]
    north = region_row.north.values[0]
    east = region_row.east.values[0]

    
    m = folium.Map(location=[  south, west,], zoom_start=14)
    folium.Rectangle(
        [[south, west], [north, east]],
        popup='The Waterfront',
        color='crimson'
    ).add_to(m)

    plt.figure(figsize=(10, 5))
    plt.plot( real_data_from_day.datetime, real_data_from_day.counts)
    plt.plot( real_data_from_day.datetime, predictions[1])
    return m

In [89]:
interact(f, region_id=regions.region.unique(), date = days_to_choose, time = time_to_choose );

interactive(children=(Dropdown(description='region_id', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 112…